In [1]:
import pandas as pd
import numpy as np
import warnings
import re
import nltk
import itertools

warnings.filterwarnings('ignore')

크롤링 파일 불러오기

In [3]:
posts = pd.read_csv('post.csv', encoding= 'unicode_escape')
comments = pd.read_csv('comment.csv')

In [49]:
print('Post shape: ', posts.shape)
print('Comments shape: ', comments.shape)

Post shape:  (90115, 363)
Comments shape:  (399029, 50)


post 데이터 전처리

In [ ]:
posts.head()

In [63]:
# 1.flair 공백 제거
posts.isnull().sum()
posts = posts[(posts.link_flair_text.notnull())]
posts.reset_index(drop=True,inplace=True)

# 2. flair 한글만 남기고 제거
posts['link_flair_text']=posts['link_flair_text'].str.replace(pat=r'[^ㄱ-ㅣ가-힣]+', repl= r'', regex=True)
posts['link_flair_text']= posts['link_flair_text'].str.replace(pat=r'[^\w]', repl=r'', regex=True)
posts['link_flair_text'].str.strip()

# 3. flair 리스트 정리
flair_list = ["문화",
                "생활",
                "경제",
                "정치",
                "자연",
                "역사",
                "레저와 취미", "레저", "취미",
                "이민",
                "건강",
                "시장",
                "개인",
                "유머",
                "범죄",
                "부고"]

# 4. posts flair 정리
posts = posts[posts['link_flair_text'].isin(flair_list)]
posts['title'] = posts['title'].apply(lambda x: re.sub('[^A-Za-z]', ' ', x))
posts = posts[~posts['selftext'].isin(['[removed]', '[deleted]'])]
posts['selftext'] = posts['selftext'].replace(np.nan, '')
# posts = posts[int(posts['num_comments']) >= 4]
posts.rename(columns={'link_flair_text': 'flair'}, inplace=True)
posts['flair'] = posts['flair'].apply(lambda x: "레저와 취미" if x=="레저" else x)
posts['flair'] = posts['flair'].apply(lambda x: "레저와 취미" if x=="취미" else x)

In [36]:
# 5. post flair 분포 확인
import matplotlib.pyplot as plt 
import seaborn as sns 

%matplotlib inline
plt.rcParams['font.family'] = 'Malgun Gothic'

posts['link_flair_text'].value_counts().plot(kind='bar')

NameError: name 'post' is not defined

In [ ]:
# 6. post reset index
posts.reset_index(drop=True,inplace=True)

In [ ]:
# 7. post 타이틀+본문
for i in range(len(posts)) : 
     if posts['selftext'].isnull()[i] == False : 
        posts.loc[i,'title'] = str(posts.iloc[i]['title']) + str(posts.iloc[i]['selftext'])
posts.drop('selftext',inplace=True, axis=1)

comment 데이터 전처리

In [ ]:
comments.head()

In [ ]:
# 1. 필요한 칼럼만 추출
comments = comments.loc[:, ['link_id', 'body', 'score', 'created_utc']]

# 2. [deleted], [removed] 등 제거
comments = comments[~comments['body'].isin(['[deleted]', '[removed]',"Thanks for posting your photograph on", "Your submission", "your submission"])]

# 3. link_id 앞 3자리 제거
comments['link_id'] = comments['link_id'].apply(lambda x: x[3:])

# 4. body에서 영문자, 숫자 빼고 제거
comments['body'] = comments['body'].str.replace('[^A-Za-z1-9]+', ' ')

# 5. body에서 null값 제거
comments = comments.dropna(subset=['body'])

포스트 별 코멘트 말뭉치 만들기

In [ ]:
# 1.딕셔너리 생성
dict = {}

for link_id, body in comments[['link_id', 'body']].values:
    if link_id in dict:
        dict[link_id].append(body)
    else:
        dict[link_id] = [body]
        
# 2. 말뭉치 생성
comments_corpus = pd.DataFrame(dict.items())
comments_corpus.columns = ['id', 'comments']
comments_corpus

# 3. 컴바인드 데이터 프레임 생성
combined = pd.merge(posts, comments_corpus, on='id', how='left')
combined = combined.dropna(subset=['comments'])
combined.reset_index(drop=True,inplace=True)
combined.head()

combined 파일 전처리

In [ ]:
def title_tokenize(post_title):

    post_text = post_title
    post_text = re.sub("[^a-zA-Z]"," ", post_text)
    post_text=post_text.lower()
    post_text=replacer.replace(post_text)
    sentence_list = sent_tokenize(post_text)
    token_sentence_list = []
    
    for sentence in sentence_list:
        word_list = word_tokenize(sentence)
        token_sentence_list.append(word_list)
        
    for token_sentence in token_sentence_list:
        for token in token_sentence:
            if token in english_stop_list:
                token_sentence.remove(token)
                
    tag_sentence_list = []
    
    for token_sentence in token_sentence_list:
        tag_list = nltk.tag.pos_tag(token_sentence)
        tag_sentence_list.append(tag_list)
        
    #stem_sentence_list = []
    
    #for tag_sentence in tag_sentence_list:
    #    print([rs.stem(word) for word, pos in tag_sentence])
    #    stem_list = [rs.stem(word) for word, pos in tag_sentence]
    #    stem_sentence_list.append(stem_list)
    
    #stem_tag_sentence_list = []
    
    #for stem_sentence in stem_sentence_list:
    #    stem_tag_list = nltk.tag.pos_tag(stem_sentence)
    #    stem_tag_sentence_list.append(stem_tag_list)
    
    for tag_sentence in tag_sentence_list:
        
        lemma_sentence = []
        
        for word, pos in tag_sentence:
            if pos in ["VB", "VBD", "VBG", "VBN", "VBP"]:
                pos='v'
                lemma_sentence.append(lemmatizer.lemmatize(word, pos=pos))
            elif pos in ["NN", "NNS", "NNP", "NNPS"]:
                pos='n'
                lemma_sentence.append(lemmatizer.lemmatize(word, pos=pos))
            elif pos in ["JJ", "JJR", "JJS"]:
                pos='a'
                lemma_sentence.append(lemmatizer.lemmatize(word, pos=pos))
            else:
                lemma_sentence.append(lemmatizer.lemmatize(word))
                
        #print("Lemma 후: ", lemma_sentence)
        return lemma_sentence

# title열 토큰화
combined['title_tokenized'] = combined['title'].apply(title_tokenize)
combined = combined[(combined.title_tokenized.notnull())]
combined.reset_index(drop=True,inplace=True)

In [ ]:
combined.to_csv('combined.csv')